# Publications markdown generator for academicpages

Takes a set of bibtex of publications and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). 

The core python code is also in `pubsFromBibs.py`. 
Run either from the `markdown_generator` folder after replacing updating the publist dictionary with:
* bib file names
* specific venue keys based on your bib file preferences
* any specific pre-text for specific files
* Collection Name (future feature)

TODO: Make this work with other databases of citations, 
TODO: Merge this with the existing TSV parsing solution

In [31]:
from pybtex.database.input import bibtex
from time import strptime
import string
import html
import os
import re

In [12]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [46]:
parser = bibtex.Parser()
bibdata = parser.parse_file("mypapers.bib")
for bib_id in bibdata.entries:
    b = bibdata.entries[bib_id].fields
    kw = b["keywords"]
    if "enclosed" in kw :

        print(b)
        
        pub_year = f'{b["year"]}'
        pub_month = "01"
        pub_day = "01"
        pub_date = pub_year+"-"+pub_month+"-"+pub_day

        title = b["title"]
        clean_title = title.replace("{", "").replace("}","").replace("\\","").replace(" ","-")      
        url_slug = re.sub("\\[.*\\]|[^a-zA-Z0-9_-]", "", clean_title)
        url_slug = url_slug.replace("--","-")
        md_filename = (str(pub_date) + "-" + url_slug + ".md").replace("--","-")
        html_filename = (str(pub_date) + "-" + url_slug).replace("--","-")


        



        auth  =bibdata.entries[bib_id].persons["author"]
        authors = ""
        print(auth)
        for i in range(len(auth)):
            p = auth[i]
            conj = ", "
            if i == len(auth) -2 :
                conj = " and "
            elif i == len(auth) -1 :
                conj = ""
        
            authors = authors+p.last_names[0]+", "+p.first_names[0][0]+"." + conj

        
        

        if 'booktitle' in b: venue = b["booktitle"] 
        elif 'journal' in b : venue = b["journal"]
        else : venue = "Preprint"


        citation = "\"" + html_escape(b["title"].replace("{", "").replace("}","").replace("\\","")) + ".\""
        citation = citation + " " + authors
        citation = citation + " " + html_escape(venue)
        citation = citation + ", " + pub_year + "."


        
        date = pub_date
        link = b["url"]



      
        md = "---\ntitle: \""   + html_escape(title.replace("{", "").replace("}","").replace("\\","")) +"\""
        md += """\npermalink: """ + "/publications/" + html_filename
        md += "\nvenue: " + "'"+html_escape(venue)+"'"
        md += "\ndate: " + str(pub_date) 
        md += "\ncollection: publications"
        md += "\npaperurl: " + "'"+link+"'"
        md += "\ncitation: " + "'"+html_escape(citation)+"'"
        md += "\n---"

        md_filename = os.path.basename(md_filename)

        with open("../_publications/" + md_filename, 'w', encoding="utf-8") as f:
            f.write(md)
        print(f'SUCESSFULLY PARSED : \"', b["title"][:60],"..."*(len(b['title'])>60),"\"")
    



OrderedCaseInsensitiveDict([('title', 'Bandits dueling on partially ordered sets'), ('booktitle', 'Advances in Neural Information Processing Systems'), ('keywords', 'enclosed, mypaper'), ('url', 'https://proceedings.neurips.cc/paper_files/paper/2017/file/995665640dc319973d3173a74a03860c-Paper.pdf'), ('pages', '2129--2138'), ('year', '2017')])
[Person('Audiffren, Julien'), Person('Ralaivola, Liva')]
SUCESSFULLY PARSED : " Bandits dueling on partially ordered sets  "
OrderedCaseInsensitiveDict([('title', 'Dichotomous Optimistic Search to Quantify Human Perception'), ('booktitle', 'International Conference on Machine Learning'), ('keywords', 'enclosed, mypaper'), ('url', 'http://proceedings.mlr.press/v139/audiffren21a/audiffren21a.pdf'), ('pages', '414--424'), ('year', '2021'), ('organization', 'PMLR')])
[Person('Audiffren, Julien')]
SUCESSFULLY PARSED : " Dichotomous Optimistic Search to Quantify Human Perception  "
OrderedCaseInsensitiveDict([('title', 'It’s not all in your feet: Improv